# Big Data Content Analytics - AUEB

## Introduction to Keras Functional API

* Lab Assistant: George Perakis
* Email: gperakis[at]aeub.gr | perakisgeorgios[at]gmail.com

### Importing Modules

In [1]:
# subroutines for fetching the CIFAR-10 dataset
from keras.datasets import cifar10 

# basic class for specifying and training a neural network
from keras.models import Model 

# basic keras layers
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten

# utilities for one-hot encoding of ground truth values
from keras.utils import np_utils 

# other imports
import numpy as np

### Set Model Hyper-Parameters

In [2]:
# in each iteration, we consider 32 training examples at once
batch_size = 32 

# we iterate up to 200 times over the entire training set (depending on the early stopping etc)
num_epochs = 200 

# Convnet Hyper-parameters

# we will use 3x3 kernels (filters) throughout the network
kernel_size = 3 

# we will use 2x2 pooling throughout the network
pool_size = 2 

# # we will initially have 32 kernels per conv. layer...
conv_depth_1 = 32 

# ...switching to 64 after the first pooling layer
conv_depth_2 = 64 

# dropout after pooling with probability 0.25
drop_prob_1 = 0.25 

# dropout in the FC layer with probability 0.5
drop_prob_2 = 0.5

hidden_size = 512 # the FC layer will have 512 neurons

### Load CIFAR-10 Dataset

* The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes
* The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. 
* There are 6,000 images of each class
* Info about the CIFAR-10 Dataset may be found here: https://en.wikipedia.org/wiki/CIFAR-10

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

170508288/170498071 [==============================] - 6s 0us/step


In [4]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 

print(f'{num_train} x {height} x {width} x {depth}')

50000 x 32 x 32 x 3


### Pre-process Images and Labels

In [5]:
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10

# calculate the number of classes
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')

# Normalise data to [0, 1] range for the train and test dataset.
X_train /= np.max(X_train) # we could also just divide with 255

X_test /= np.max(X_test) # Normalise data to [0, 1] range

# One-hot encode the labels for the train and test dataset
Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 

### Build Convolutional Model using Functional API

In [6]:
inp = Input(shape=(height, width, depth))

# set the convolutional layer 1 (conv_1)
conv_1 = Convolution2D(conv_depth_1,  # this is just the instantiation. 
                       (kernel_size, kernel_size),
                       padding='same',
                       activation='relu')

# pass the input layer through the first conv-layer
conv_1 = conv_1(inp)

# set the convolutional layer 2 (conv_2)
conv_2 = Convolution2D(conv_depth_1, # this is NOT just the instantiation. 
                       (kernel_size, kernel_size),
                       padding='same', 
                       activation='relu')(conv_1) # in this line we pass the conv_1 as input to conv_2

# Create a Max Pooling Layer for the 2D filter 
pool_1 = MaxPooling2D(pool_size=(pool_size,
                                 pool_size))(conv_2) # pass here the second conv-layer (conv_2)

# Add a Dropout Layer to reduce overfitting
drop_1 = Dropout(drop_prob_1)(pool_1)

# Create a third conv-layer
conv_3 = Convolution2D(conv_depth_2,
                       (kernel_size, kernel_size),
                       padding='same',
                       activation='relu')(drop_1)

# Create a fourth conv-layer
conv_4 = Convolution2D(conv_depth_2,
                       (kernel_size, kernel_size),
                       padding='same', 
                       activation='relu')(conv_3)

# Again a Max Pooling Layer
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)

# Another dropout layer after the max-pooling layer
drop_2 = Dropout(drop_prob_1)(pool_2)

# Flatten the output of the drop_2
flat = Flatten()(drop_2)

# Add a Dense layer
hidden = Dense(hidden_size, 
               activation='relu')(flat)

# Add a Dropout layer
drop_3 = Dropout(drop_prob_2)(hidden)

# We use 10 classes, with the SOFT-MAX activation function.
out = Dense(num_classes,
            activation='softmax')(drop_3)

# create the actual model
model = Model(inputs=inp, outputs=out)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928 

### Compile Model

In [8]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) 

### Set Model Callbacks

In [9]:
from tensorflow.python.keras.callbacks import EarlyStopping

# setting the early stoping callback
es = EarlyStopping(monitor='val_loss',
                   patience=4,
                   verbose=1,
                   restore_best_weights=True)

### Model Fit

In [ ]:
# Train the model using the training set...
model.fit(X_train, 
          Y_train,                
          batch_size=batch_size, 
          epochs=num_epochs,
          verbose=1,
          validation_split=0.1, # ...holding out 10% of the data for validation
          callbacks=[es]) 


Epoch 1/200
1407/1407 [==============================] - 62s 13ms/step - loss: 1.7920 - accuracy: 0.3361 - val_loss: 1.1299 - val_accuracy: 0.6032
Epoch 2/200
1407/1407 [==============================] - 17s 12ms/step - loss: 1.1775 - accuracy: 0.5746 - val_loss: 1.0292 - val_accuracy: 0.6398
Epoch 3/200
 880/1407 [=================>............] - ETA: 6s - loss: 1.0265 - accuracy: 0.6362

### Model Evaluation

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

313/313 [==============================] - 1s 3ms/step - loss: 0.6727 - accuracy: 0.7774


[0.6726597547531128, 0.777400016784668]